# **Introducao a Analise Preditiva**

**prof: Sergio Assuncao Monteiro, DSc**

linkedin: https://www.linkedin.com/in/sergio-assun%C3%A7%C3%A3o-monteiro-b781897b/

lattes: http://lattes.cnpq.br/9489191035734025


# **Instalação do pacote mip**

documentação do pacote MIP: https://python-mip.readthedocs.io/en/latest/classes.html

In [6]:
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Problema da Mochila**

Formulação do Modelo do problema da mochila (knapsack problem)

\begin{eqnarray}
\max \ 10\ x_{1} &+ 13\ x_{2}+18\ x_{3}+31\ x_{4}+7\ x_{5}+15\ x_{6}  \\
\mbox{sujeito a:}&  \\
11\ x_{1} + 15\ x_{2} &+20\ x_{3}+35\ x_{4}+10\ x_{5}+33\ x_{6} \leq 53  
\end{eqnarray}


Além disso $x$ é um vetor inteiro com valores não-negativos.

In [7]:
from mip import Model, maximize, xsum, CBC, BINARY, OptimizationStatus

In [8]:
coef_funcao_objetivo = [10, 13, 18, 31, 7, 15]
coef_restr = [11, 15, 20, 35, 10, 33]
termo_independente = 53 

In [9]:
I = range(len(coef_funcao_objetivo))
m = Model("knapsack")

In [10]:
x = [m.add_var(var_type=BINARY) for i in I]

In [11]:
m.objective = maximize(xsum(coef_funcao_objetivo[i] * x[i] for i in I))

In [12]:
m += xsum(coef_restr[i] * x[i] for i in I) <= termo_independente
print(f'O modelo tem {m.num_cols} variável(eis), {m.num_rows} restrição(ões) e {m.num_nz} zero(s)')

O modelo tem 6 variável(eis), 1 restrição(ões) e 6 zero(s)


In [13]:
status = m.optimize(max_seconds=2)

In [14]:
status == OptimizationStatus.OPTIMAL

True

In [15]:
itens_selecionados = ["x"+str(i+1) for i in I if x[i].x >= 0.99]
print("Itens selecionados: {}".format(itens_selecionados))

Itens selecionados: ['x2', 'x4']


# **Instalação do pacote ortools**

In [16]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 KB 45.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.
tensorflow-metadata 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.12 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.


**Exemplo de Programação Inteira**

In [17]:
from ortools.linear_solver import pywraplp

In [18]:
solver = pywraplp.Solver.CreateSolver('SCIP')

In [19]:
infinity = solver.infinity()
# x e y são variáveis inteiras não-negativas.
x = solver.IntVar(0.0, infinity, 'x')
y = solver.IntVar(0.0, infinity, 'y')

print('Número de variáveis =', solver.NumVariables())

Número de variáveis = 2


In [20]:
# x + 7 * y <= 17.5.
solver.Add(x + 7 * y <= 17.5)

# x <= 3.5.
solver.Add(x <= 3.5)

print('Número de restrições =', solver.NumConstraints())

Número de restrições = 2


In [21]:
# Maximize x + 10 * y.
solver.Maximize(x + 10 * y)

In [22]:
status = solver.Solve()

In [23]:
if status == pywraplp.Solver.OPTIMAL:
  print('Solução:')
  print('Valor objetivo =', solver.Objective().Value())
  print('x =', x.solution_value())
  print('y =', y.solution_value())
else:
  print('O problema não tem solução ótima.')



Solução:
Valor objetivo = 23.0
x = 3.0
y = 2.0


In [24]:
print('\nInformações sobre o processamento:')
print(f'Tempo de processamento em milisegundos foi de {solver.wall_time()}')
print(f'Quantidade de Iterações para resolver o problema foi de {solver.iterations()}')
print(f'Quantidade nós de branch-and-bound para resolver o problema foi de {solver.nodes()}')


Informações sobre o processamento:
Tempo de processamento em milisegundos foi de 59
Quantidade de Iterações para resolver o problema foi de 0
Quantidade nós de branch-and-bound para resolver o problema foi de 1
